In [87]:
import pandas as pd
import numpy as np
import logging
import sys
from datetime import datetime
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import scipy
import copy
from scipy.stats import skewnorm
from random import expovariate

# a little hacky, but works if you don't want to actually install the
# custom packages 
sys.path.append('..')
from uniswapv3_simulator.pool import Uniswapv3Pool
from uniswapv3_simulator.utils import pool_init_price, solve_for_liquidity_delta
from uniswapv3_simulator.math import tick_to_sqrt_price, sqrt_price_to_tick
from utils import amount_to_float

expt = 'simulation_v1'
timestamp = datetime.now().strftime('%y%m%d%H%M%S')
logging.basicConfig(level=logging.INFO,
                    filename=f'./{expt}_{timestamp}.log',
                    )
logging.getLogger('uniswap-v3').setLevel(logging.DEBUG)
logging.getLogger('covalent_api').setLevel(logging.DEBUG)

logger = logging.getLogger('experiment')

In [89]:
from uniswapv3_simulator.utils import sqrt_price_to_tick
from uniswapv3_simulator.tick import MAX_TICK, MIN_TICK

init_price = 3088
fee = 0.03
budget = 10000
num_ticks = 10000

init_tick = sqrt_price_to_tick(np.sqrt(init_price))
ticks = np.round(np.linspace(init_tick + 1, MAX_TICK, num_ticks), 0)

In [90]:
def init_uniform_pool(fee, price, budget, tick_spacing = 1):
    pool = Uniswapv3Pool(fee, tick_spacing, price)
    pool.set_position('uniform', MIN_TICK, MAX_TICK, budget)
    return pool

In [92]:
_pool = init_uniform_pool(fee, init_price, budget, tick_spacing = 1)
tick = ticks[0]
txn_rate = 100
alpha = 0.5
num_sims = 10**5

In [118]:
def simulate(_pool, tick, txn_rate, alpha, num_sims, budget, txn_modifier=60 * 60, mu=0, sigma=0.1):
    """
        pool: already instantiated with Liquidity shape and depth
        tick: p2 of the range we are considering, >= p
        txn_rate: transactions per unit; i.e. 100 txn per hour requires txn_modifier = 60 * 60
        alpha: number of arbs
        budget: budget for liquidity
        txn_modifier: factor to convert from txn units to seconds, 60 * 60 is for hours
        
        mu: drift for GBM
        sigma: vol for GBM
    """
    # copy pool object
    pool = copy.deepcopy(_pool)
    pool_tick = pool.tick
    price = pool.sqrt_price ** 2
    
    p2 = tick
    p1 = 2 * pool_tick - p2

    # we have our symmetric range: (p1, p2) which surrounds the initial price
    # now add our position
    pool.set_position('target', p1, p2, budget)
    fees = np.empty(num_sims)
    for i in range(num_sims):
        # draw arrival times
        arrivals = np.random.exponential(1.0 / (txn_rate / txn_modifier), int(2.5 * txn_rate))
        cumulative_arrival = np.cumsum(arrivals) / txn_modifier
        arrivals = arrivals[cumulative_arrival <= 1.0] / txn_modifier
        
        for dt in arrivals:
            u, n = np.random.uniform(), np.random.normal()
            X = (mu - 0.5 * sigma ** 2)*dt + (sigma * np.sqrt(dt) * n)
            new_price = price * np.exp(X)
            if u < alpha:
                # this is an arbitrage, trade to new price
                price = new_price
                pool.swap(price)
            else:
                # this is a liquidity trade, trade to new price and back
                pool.swap(new_price)
                pool.swap(price)
        fees[i] = pool.get_position('target')
    
    return np.mean(fees), np.std(fees)
    
simulate(_pool, tick, txn_rate, alpha, num_sims, budget, txn_modifier=3600)

(0.011728659863466606, 0.06196822719186202)